In [1]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")
using DataStructures
using DataStructuresExtended
using JLD
using DataFrames
using Gadfly

In [2]:
function groupby_value(fn::Function, xs)
    key_type = typeof(fn(first(xs)))
    group_type = Vector{typeof(first(xs))}
    groups = Dict{key_type, group_type}()
    
    for x in xs
        key = fn(x)
        group = get!(()->group_type(), groups, key)
        push!(group, x)
    end
    groups
end
function round_tbl(data::DataFrame, places=2)
    ret = copy(data)
    for (ii,column) in enumerate(ret.columns)
        if typeof(column[1])<:Number #hack
            ret.columns[ii] = round(column, places)
        end
    end
    ret
end


round_tbl (generic function with 2 methods)

In [3]:
function flatten_blocked_records{T<:Number, V}(records::Dict{T,V})  
    rec_types = @pipe records |> values |> map(typeof,_) |> union
    rec_type = length(rec_types)==1 ? rec_types[1] : V
    ret = rec_type() #Construct it, it is some form of vector, or we will error soon
    sizehint!(ret) = sum([length(r) for r in values(records)])
    
    sorted_keys = @pipe records |> keys |> collect |> sort
    for key in sorted_keys
        push!(ret, records[key]...)
    end
    ret
end

function flatten_raw_blocked_records{T,V}(records::Dict{T,V})  
    key_types = @pipe records |> keys|> map(parse,_) |> map(typeof,_) |> union
    @assert length(key_types)==1
    key_type =  key_types[1]
    
    rec_types = @pipe records |> values |> map(typeof,_) |> union
    rec_type = length(rec_types)==1 ? rec_types[1] : V
    flatten_blocked_records(Dict(Pair{key_type, rec_type}[parse(k)=>v for (k,v) in records]))
end

flatten_raw_blocked_records (generic function with 1 method)

In [4]:
OUTPUT_DIR = "../../documents_prepared/Journal\ Papers/SumResynthesis/data/"

"../../documents_prepared/Journal Papers/SumResynthesis/data/"

In [ ]:
######Unordered/BOW

In [5]:
function jaccard_index(aa::DataStructures.Accumulator,bb::DataStructures.Accumulator)
    sum(intersect(aa,bb))/sum(union(aa,bb))
end

function jaccard_index(aa,bb)
    length(intersect(aa,bb))/length(union(aa,bb))
end

jaccard_index (generic function with 2 methods)

In [21]:

#Bag based definitions
#Itersection of two bags has all elements of each with multiplicity equal to the lower.
function precision(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
    sum(intersect(actual,reference))/sum(actual)
end

function recall(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
    sum(intersect(actual,reference))/sum(reference)
end

#Set based definitions
#function precision(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
#    actual= Set(keys(actual))
#    reference= Set(keys(reference))
#    length(intersect(actual,reference))/length(actual)
#end
#function recall(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
#    actual= Set(keys(actual))
#    reference= Set(keys(reference))
#    length(intersect(actual,reference))/length(reference)
#end

#Overall

function f1_score(actual::DataStructures.Accumulator,reference::DataStructures.Accumulator)
    prec = precision(actual,reference)
    rec = recall(actual,reference)
    2*prec*rec/(prec+rec)
end

f1_score (generic function with 1 method)

f1_score (generic function with 1 method)

In [6]:

#books_glove300cityblock_raw = load("results/bags/books_corpus_0.001_of_test_glove300_cityblock.jld","4")
books_0_01_glove300_raw = flatten_raw_blocked_records(load("results/bags/books_corpus_0.01_of_test.jld"))
#books_001o_glove300_raw = flatten_raw_blocked_records(load("results/bags/books_corpus_0.01_of_test_glove300.jld"))
#brown_glove300_raw = load("results/bags/brown_glove300_res.jld", "res")
#brown_glove200_raw = load("results/bags/brown_glove200_res.jld", "res")
#brown_glove100_raw = load("results/bags/brown_glove100_res.jld", "res")
#brown_glove50_raw = load("results/bags/brown_glove50_res.jld", "res")

66464-element Array{Tuple{Array{Symbol,1},Array{Symbol,1},Float32},1}:
 ([:okay,symbol(","),:i,:said,symbol(","),:this,:is,:going,:to,:be,:some,:fun,:revenge,:.],[:i,:it,:be,:.,symbol(","),:going,:revenge,:some,symbol(","),:brainchild,symbol("74.9"),:go,:very,:also,symbol("'ll"),:adding],-8.218232f0)                                                                                                                         
 ([:this,:is,:just,:great,symbol(","),:seth,:shouted,:.],[:this,:is,:.,:great,:shouted,:seth,symbol(","),:just],-1.4714614f-6)                                                                                                                                                                                                                                 
 ([:start,:in,:the,:beginning,:and,:do,symbol("n't"),:hold,:anything,:back,:.],[symbol("n't"),:do,:in,:back,:start,:the,:.,:hold,:and,:beginning,:anything],-3.0723756f-6)                                                       

In [ ]:
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove300_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove200_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove100_raw])

In [14]:
bag_res = DataFrame()
#bag_res[:ground] = [counter(rset[1]) for rset in brown_glove50_raw]
bag_res[:ground] = [counter(rset[1]) for rset in  books_0_01_glove300_raw]
bag_res[:ground_len] = Int[sum(ss) for ss in bag_res[:ground]]
exp_names = ASCIIString[]
#for raw_name in ["brown_glove50_raw","brown_glove100_raw", "brown_glove200_raw", "brown_glove300_raw"]#
for raw_name in ["books_0_01_glove300_raw"]
    colname = join(split(raw_name,"_")[1:end-1],"_")
    push!(exp_names, colname)
    raw = eval(raw_name|>Symbol)
    bag_res[colname*"_actual"|> Symbol] = [counter(rset[2]) for rset in raw]
    #bag_res[colname*"_distance"|> Symbol] = Float64[-1*rset[3] for rset in raw]
end

#bag_res

In [15]:
for exp_name in exp_names
    actual = bag_res[exp_name*"_actual" |> symbol]
    bag_res[exp_name*"_perfect" |> symbol] = actual.==bag_res[:ground]
    bag_res[exp_name*"_jaccard" |> symbol] = map(jaccard_index, actual, bag_res[:ground])
    bag_res[exp_name*"_precision" |> symbol] = map(precision, actual, bag_res[:ground])
    bag_res[exp_name*"_recall" |> symbol] = map(recall, actual, bag_res[:ground])
    bag_res[exp_name*"_f1" |> symbol] = map(f1_score, actual, bag_res[:ground])
end

In [16]:
keep_cols = @pipe (bag_res.colindex |> keys |> map(string,_)
                    |>filter(col -> contains(col,"jaccard") 
                                ||  contains(col,"perfect") 
                                ||  contains(col,"recall")
                                ||  contains(col,"_precision")
                                ||  contains(col,"f1" ),_)
                    |> map(Symbol,_))
len_scores = aggregate(bag_res[[:ground_len, keep_cols...]], :ground_len, mean)
sort!(len_scores)
#writetable(OUTPUT_DIR*"selection_len_scores.csv", len_scores)
len_scores |> round_tbl

,ground_len,books_0_01_glove300_perfect_mean,books_0_01_glove300_jaccard_mean,books_0_01_glove300_precision_mean,books_0_01_glove300_recall_mean,books_0_01_glove300_f1_mean
1,1,1.0,1.0,1.0,1.0,1.0
2,2,1.0,1.0,1.0,1.0,1.0
3,3,1.0,1.0,1.0,1.0,1.0
4,4,1.0,1.0,1.0,1.0,1.0
5,5,0.99,0.99,1.0,1.0,1.0
6,6,0.98,0.99,0.99,0.99,0.99
7,7,0.97,0.99,0.99,0.99,0.99
8,8,0.95,0.98,0.99,0.99,0.99
9,9,0.94,0.98,0.99,0.99,0.99
10,10,0.92,0.98,0.98,0.99,0.99


In [ ]:
#writetable(OUTPUT_DIR*"selection_len_scores_brown.csv", len_scores)

In [17]:
cum_len_scores = DataFrame()
cum_len_scores[:max_ground_len] = bag_res[:ground_len] |> unique |> sort!
for col in keep_cols
    cum_len_scores[col] = Float64[ begin
        shortish_sents = bag_res[:ground_len] .<= max_len;
        shortish_bag_res = bag_res[shortish_sents, col]
        shortish_bag_res[isfinite(shortish_bag_res)] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end

writetable(OUTPUT_DIR*"selection_cum_len_scores_books.csv", cum_len_scores)
cum_len_scores |> round_tbl


,max_ground_len,books_0_01_glove300_perfect,books_0_01_glove300_jaccard,books_0_01_glove300_precision,books_0_01_glove300_recall,books_0_01_glove300_f1
1,1,1.0,1.0,1.0,1.0,1.0
2,2,1.0,1.0,1.0,1.0,1.0
3,3,1.0,1.0,1.0,1.0,1.0
4,4,1.0,1.0,1.0,1.0,1.0
5,5,0.99,1.0,1.0,1.0,1.0
6,6,0.99,1.0,1.0,1.0,1.0
7,7,0.98,0.99,1.0,1.0,1.0
8,8,0.98,0.99,0.99,1.0,1.0
9,9,0.97,0.99,0.99,0.99,0.99
10,10,0.97,0.99,0.99,0.99,0.99


In [18]:
cum_len_scores[end,:] 

,max_ground_len,books_0_01_glove300_perfect,books_0_01_glove300_jaccard,books_0_01_glove300_precision,books_0_01_glove300_recall,books_0_01_glove300_f1
1,246,0.7556120606644199,0.8906838509159689,0.913871336385951,0.9351272214152089,0.9231796594020278


In [ ]:
function longform(df::DataFrame, index::Symbol, cols::Vector{Symbol}, new_name::Symbol)
    function subdf(col::Symbol)
        df_sub = df[[index, col]]
        rename!(df_sub, col, new_name)
        df_sub[:label] = col
        df_sub
    end
    vcat([subdf(col) for col in cols]...)
end

In [ ]:
using Gadfly

In [ ]:
p=Gadfly.plot(longform(cum_len_scores, :max_ground_len, jaccard_cols, :mean_jaccard_index),
    x="max_ground_len", y="mean_jaccard_index",
    color="label",
    Coord.Cartesian(ymin=0),
    Geom.point
)
#draw(PNG("brown_cumulative_length_score.png", 24cm, 6cm), p)

In [ ]:

p=Gadfly.plot(longform(len_scores, :ground_len, [string(col)*"_mean"|> Symbol for col in jaccard_cols], :mean_jaccard_index),
    x="ground_len", y="mean_jaccard_index",
    color="label",
    Geom.point
)
#draw(PNG("brown_length_score.png", 24cm, 6cm), p)

In [ ]:
bag_res[:ground_len] |> describe 
p=Gadfly.plot(bag_res, x="ground_len",  Geom.histogram)
#draw(PNG("brown_length_dist.png", 24cm, 6cm), p)

In [ ]:
##############################################
# ORDERED
########################################

In [26]:
using PyCall
@pyimport nltk
@pyimport nltk.align.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(candidate,Any[reference], weights)
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


bleu_score (generic function with 1 method)

In [27]:
bleu_score(["the","comedy","of","manners","of","this","first","a","basis","is","performed","in","1892","."],
           ["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."]
)

0.30576902884505114

In [28]:
immutable OrderRecord{T}
    reference::Vector{T}
    actual::Vector{T}
    prob::Float64
    time::Float64
    status::Symbol
end

In [29]:
function load_ordering_results(path, keys)
    blocked_records = Dict{Int,Any}([key=>load(path,string(key)) for key in keys])
    @pipe blocked_records |> flatten_blocked_records |> map(r->OrderRecord(r...),_)
end

#books_0_001 = load_ordering_results("results/ordered/books_corpus_0.001_of_test_glove300.jld", 1:100)
#books_0_001_oracle = load_ordering_results("results/ordered/books_corpus_0.001_of_test_oracle.jld", 1:100)

load_ordering_results (generic function with 1 method)

In [30]:
books_0_01 = load_ordering_results("results/ordered/books_corpus_0.01_of_test.jld", 1:1000)
books_0_01_oracle = load_ordering_results("results/ordered/books_corpus_0.01_of_test_oracle.jld", 1:1000)

66464-element Array{OrderRecord{Symbol},1}:
 OrderRecord{Symbol}([:okay,symbol(","),:i,:said,symbol(","),:this,:is,:going,:to,:be,:some,:fun,:revenge,:.],[:okay,symbol(","),:i,:said,symbol(","),:this,:is,:going,:to,:be,:some,:fun,:revenge,:.],1.2773596628139228e-22,38.715032542,:Optimal)                                                                                                                                       
 OrderRecord{Symbol}([:this,:is,:just,:great,symbol(","),:seth,:shouted,:.],[:seth,:shouted,symbol(","),:this,:is,:just,:great,:.],1.252549565980782e-15,5.772037558,:Optimal)                                                                                                                                                                                                             
 OrderRecord{Symbol}([:start,:in,:the,:beginning,:and,:do,symbol("n't"),:hold,:anything,:back,:.],[:do,symbol("n't"),:hold,:anything,:back,:and,:start,:in,:the,:beginning,:.],4.550069306365155e-20

In [31]:
attempted_only{T<:OrderRecord}(recs::Vector{T}) = filter(r->r.status!=:NotAttempted, recs)
feasible_only{T<:OrderRecord}(recs::Vector{T}) = filter(r->r.status!=:Infeasible, recs)


feasible_only (generic function with 1 method)

In [32]:
function exact_match(rec::OrderRecord)
    rec.reference == rec.actual
end
function bleu_score(rec::OrderRecord)
    bleu_score( rec.actual, rec.reference)
end

bleu_score (generic function with 2 methods)

In [55]:
eval_methods = [
    #(symbol("Time Taken (s)"), data -> map(r->r.time , data|> attempted_only)),
    (symbol("Portion Perfect"), data ->  map(exact_match, data|> attempted_only)),
    (symbol("BLEU Score"), data ->     map(bleu_score,  data|> attempted_only)),
    (symbol("Portion Feasible"), data ->     map(r->r.status==:Optimal, data|> attempted_only)),
    (symbol("Portion Perfect (Feasible Only)"), data ->  map(exact_match, data|> attempted_only |>feasible_only)),
    (symbol("BLEU Score (Feasible Only)"), data ->     map(bleu_score,  data|> attempted_only |>feasible_only) ),
]

5-element Array{Tuple{Symbol,Function},1}:
 (symbol("Portion Perfect"),(anonymous function))                
 (symbol("BLEU Score"),(anonymous function))                     
 (symbol("Portion Feasible"),(anonymous function))               
 (symbol("Portion Perfect (Feasible Only)"),(anonymous function))
 (symbol("BLEU Score (Feasible Only)"),(anonymous function))     

In [45]:
books_0_01 |> eval_methods[2][2] |> mean

0.6216002261803788

In [59]:
function get_ordering_res(datas...)
    ordering_res = DataFrame(Process=[model for (model, data) in datas])
    for (method, fn) in eval_methods
        println(method)
        ordering_res[method]=[mean(fn(data)) for (model, data) in datas]
    end
    ordering_res
end

ordering_res = get_ordering_res(("Ordering",books_0_01_oracle),("Two Step",books_0_01))

writetable(OUTPUT_DIR*"ordering_scores.csv", ordering_res)
round_tbl(ordering_res)

Portion Perfect
BLEU Score
Portion Feasible
Portion Perfect (Feasible Only)
BLEU Score (Feasible Only)


,Process,Portion Perfect,BLEU Score,Portion Feasible,Portion Perfect (Feasible Only),BLEU Score (Feasible Only)
1,Ordering,0.67,0.81,1.0,0.67,0.81
2,Two Step,0.62,0.75,0.94,0.66,0.79


In [ ]:
function mean_by_length{T<:OrderRecord}(rater::Function, data::Vector{T}, assessed_lengths)
    groups = groupby_value(r->length(r.reference), data) #r.reference is the ground truth sentence
    [ mean(rater(groups[len])) for len in assessed_lengths]
end
function get_instance_counts(data)
    groups = groupby_value(r->length(r.reference), data) 
    [length(groups[sent_len]) for sent_len in sort(collect(keys(groups)))]
end

function get_ordering_length_res(data)
    ordering_length_res = DataFrame(
        ground_length=1:maximum([length(r.reference) for r in data |> attempted_only]),
        Instances = get_instance_counts(data|> attempted_only))
    for (method, fn) in eval_methods
        println(method)
        ordering_length_res[method]=mean_by_length(fn,data, ordering_length_res[:ground_length])
    end
    return ordering_length_res

end
ordering_length_res = get_ordering_length_res(books_0_01)
writetable(OUTPUT_DIR*"ordering_length_scores.csv", ordering_length_res)
round_tbl(ordering_length_res)

In [ ]:
ordering_length_res_oracle = get_ordering_length_res(books_0_01_oracle)
writetable(OUTPUT_DIR*"ordering_length_scores_oracle.csv", ordering_length_res_oracle)
round_tbl(ordering_length_res_oracle)

In [22]:
# Compare Errors to those from Selection

In [23]:
oracle_bleu = ordering_length_res_oracle[:,symbol("BLEU Score")]
generated_bleu = ordering_length_res[:,symbol("BLEU Score")]
selection_prec = len_scores[1:18,:books_0_01_glove300_precision_mean]

predicted_bleu = oracle_bleu-(1-selection_prec) #Expected loss from word ordering loss can be removed
print([predicted_bleu generated_bleu])

Gadfly.plot(layer(x=1:18, y=oracle_bleu,Geom.line, Geom.point, Theme(default_color=color("green"))),
            layer(x=1:18, y=predicted_bleu,Geom.line, Geom.point),
            layer(x=1:18, y=generated_bleu,Geom.line, Geom.point,Theme(default_color=color("red")))
)

LoadError: LoadError: UndefVarError: ordering_length_res_oracle not defined
while loading In[23], in expression starting on line 1

In [24]:
keys(ordering_length_res_oracle)

LoadError: LoadError: UndefVarError: ordering_length_res_oracle not defined
while loading In[24], in expression starting on line 1

In [25]:
oracle_exact = ordering_length_res_oracle[:,symbol("Portion Perfect")]
generated_exact = ordering_length_res[:,symbol("Portion Perfect")]
selection_exact = len_scores[1:18,:books_0_01_glove300_perfect_mean]

predicted_exact = oracle_exact-(1-selection_exact) #Expected loss from word ordering loss can be removed
print([predicted_bleu generated_bleu])

Gadfly.plot(layer(x=1:18, y=oracle_exact,Geom.line, Geom.point, Theme(default_color=color("green"))),
layer(x=1:18, y=predicted_exact,Geom.line, Geom.point),
layer(x=1:18, y=generated_exact,Geom.line, Geom.point,Theme(default_color=color("red")))
)

LoadError: LoadError: UndefVarError: ordering_length_res_oracle not defined
while loading In[25], in expression starting on line 1

In [ ]:
Gadfly.plot(ordering_length_res, x=:ground_length, y=:BLEU_Score)

In [ ]:
using Gadfly

In [ ]:
p=Gadfly.plot(ordering_length_res, x=:ground_length, y=:Time_taken)
#draw(PNG("ordering_time_taken_vs_length.png", 10cm, 6cm), p)

In [ ]:
function cum_means(indexed_data::DataFrame, index_col=:ground_length, count_col=:Instances)
    @assert issorted(indexed_data[index_col])
    cum_data=DataFrame()
    cum_data[Symbol("max_"*string(index_col))] = indexed_data[index_col]
    cum_data[count_col] = cumsum(indexed_data[count_col])
    keep_cols = @pipe indexed_data.colindex |> keys |> collect |> filter(col->col∉[index_col, count_col], _) |> collect
    for col in keep_cols
        total_count = 0
        total_score = 0
        prev_index = 0 
        function cum(ii)
            @assert ii>prev_index
            score = indexed_data[ii,col]
            count = indexed_data[ii,count_col]
            total_score = (total_score*total_count+score*count)/(total_count+count)
            total_count+=count
            prev_index = ii
            return total_score
        end
        cum_data[col]= map(cum, 1:length(indexed_data[col]))
    end
    cum_data
end
    

In [ ]:
cum_means(ordering_length_res)

In [ ]:
cum_means(ordering_length_res_oracle)

In [ ]:
delete!()

In [ ]:
########
#Examples

In [60]:
examples = load_ordering_results("results/ordered/books_corpus_examples_glove300.jld", 1:1)
examples_oracle = load_ordering_results("results/ordered/books_corpus_examples_glove300_oracle.jld", 1:1)

10-element Array{OrderRecord{Symbol},1}:
 OrderRecord{Symbol}([:name,:this,symbol("1922"),:novel,:about,:leopold,:bloom,:written,:by,:james,:joyce,:.],[:written,:by,:name,:this,:.,:novel,:about,symbol("1922"),:bloom,:leopold,:james,:joyce],NaN,4.735941975,:Infeasible)                                                                                                                                                              
 OrderRecord{Symbol}([:ralph,:waldo,:emerson,:dismissed,:this,:poet,:as,:the,:jingle,:man  …  :lowell,:called,:him,symbol("three-fifths"),:genius,:and,symbol("two-fifths"),:sheer,:fudge,:.],[:sheer,:this,:as,:james,symbol("two-fifths"),:emerson,:fudge,:lowell,:poet,:genius  …  :the,:and,:ralph,:and,:him,:.,:dismissed,:jingle,symbol("three-fifths"),:man],NaN,18.379483723,:Infeasible)
 OrderRecord{Symbol}([:this,:is,:the,:basis,:of,:a,:comedy,:of,:manners,:first,:performed,:in,symbol("1892"),:.],[:this,:is,:the,:basis,:of,:a,:comedy,:of,:manners,:first,:performed,:in,s

In [61]:
IyyerTreeProvidedTitle = "\\iref"
IyyerTreeAlternateTitle = "\\ip"
BowmanMean = "\\bm"
BowmanSample = "\\bs"
crossrefs = Any[
    (
        (IyyerTreeProvidedTitle ,  "name this 1906 novel about gottlieb_fecknoe inspired by james_joyce"),
         (IyyerTreeAlternateTitle ,  "what is this william golding novel by its written writer"),
    ),
    (
        (IyyerTreeProvidedTitle ,  "henry_david_thoreau rejected this author like the tsar boat and imbalance created known good writing and his own death"),
        (IyyerTreeAlternateTitle ,  "henry_david_thoreau rejected him through their stories to go money well inspired stories to write as her writing"),
    ),
    (
        (IyyerTreeProvidedTitle ,  "another is the subject of this trilogy of romance most performed in 1874"),
        (IyyerTreeAlternateTitle ,  "subject of drama from him about romance "),
    ),
    (
        (IyyerTreeProvidedTitle ,  "during the short book the lady seduces the family and meets cousin he in a novel dies sister from the mr."),
        (IyyerTreeAlternateTitle ,  "during book of its author young lady seduces the family to marry old suicide while i marries himself in marriage"),
    ),
    (
        (IyyerTreeProvidedTitle ,  "however she leaves her sister and daughter from former fianc and she ends unfortunately when narrator drives into life of a house"),
        (IyyerTreeAlternateTitle ,  "leaves the sister of man in this novel"),
    ),
    (
        (BowmanMean, "they were laughing at the same time ."),
        (BowmanSample*" 1", "ill see you in the early morning ."),
        (BowmanSample*" 2",  "i looked up at the blue sky ."),
        (BowmanSample*" 3", "it was down on the dance floor .")
    ),
    (
        (BowmanMean, "i went to the kitchen ."),
        (BowmanSample*" 1", "i went to my apartment . "),
        (BowmanSample*" 2",  "i looked around the room ."),
        (BowmanSample*" 3", "i turned back to the table .")
    ),
    (
        (BowmanMean, "what are you doing ?"),
        (BowmanSample*" 1", "“ are you sure ?"),
        (BowmanSample*" 2",  "what are you doing, ?"),
        (BowmanSample*" 3", "what are you doing ?"),
    ),
    [],
    []
]

10-element Array{Any,1}:
 (("\\iref","name this 1906 novel about gottlieb_fecknoe inspired by james_joyce"),("\\ip","what is this william golding novel by its written writer"))                                                                                                           
 (("\\iref","henry_david_thoreau rejected this author like the tsar boat and imbalance created known good writing and his own death"),("\\ip","henry_david_thoreau rejected him through their stories to go money well inspired stories to write as her writing"))
 (("\\iref","another is the subject of this trilogy of romance most performed in 1874"),("\\ip","subject of drama from him about romance "))                                                                                                                      
 (("\\iref","during the short book the lady seduces the family and meets cousin he in a novel dies sister from the mr."),("\\ip","during book of its author young lady seduces the family to marry old

In [69]:
counter([1,2,3])==counter([2,3,1])

true

In [76]:
begin_tabular = "\\begin{tabular}{ p{\\collenone} p{\\collentwo} p{\\collenthree} p{\\collenthree} }\n"
end_tabular = "\\end{tabular}"


    
println(begin_tabular)
ids = "3A 3B 3C 3D 3E 4A 4B 4C 5A 5B" |> split

for (row_ii, (id, genned, oracle,crossref)) in enumerate(zip(ids, examples, examples_oracle,crossrefs))
    function print_latex_row(field, vals...)
        print("\\textbf{$field} ")
        for val in vals
            val = replace(val, "_","\\_")
            print(" & ",val)
        end
        println(" \\\\")
    end
    format(ws) = join(map(string, ws)," ")
    
    function get_stats(record, oracle::Bool)
        
        bow = oracle ? "--" :
              counter(record.actual) == counter(record.reference) ? "\\cmark" :
              "\\xmark"
              
        
        ord = record.status==:Infeasible ? "\\namark" : 
              record.actual == record.reference ? "\\cmark" :
              "\\xmark"
        
        bow, ord
    end
    
    
    print_latex_row("$id \\hfill \\reftitle", format(genned.reference),"\\textbf{Sel.}","\\textbf{Ord.}")
    print_latex_row("\\oracletitle", format(oracle.actual), get_stats(oracle,true)...)
    print_latex_row("\\twosteptitle", format(genned.actual), get_stats(genned,false)...)
    for (field, sentence) in crossref
        print_latex_row(field, sentence,"","")
    end
    if (row_ii==5)
        println(end_tabular)
        println("\n\n\n\n\b")
        println(begin_tabular)
    elseif row_ii==8
        println(end_tabular)
        println("\n\n\n\n\b")
        println(begin_tabular)
    else 
        println("\\hline")
    end
    
    
end
println(end_tabular)

\begin{tabular}{ p{\collenone} p{\collentwo} p{\collenthree} p{\collenthree} }

\textbf{3A \hfill \reftitle}  & name this 1922 novel about leopold bloom written by james joyce . & \textbf{Sel.} & \textbf{Ord.} \\
\textbf{\oracletitle}  & written by name this . novel about 1922 bloom leopold james joyce & -- & \namark \\
\textbf{\twosteptitle}  & written novel by name james about leopold this bloom 1922 joyce . & \cmark & \namark \\
\textbf{\iref}  & name this 1906 novel about gottlieb\_fecknoe inspired by james\_joyce &  &  \\
\textbf{\ip}  & what is this william golding novel by its written writer &  &  \\
\hline
\textbf{3B \hfill \reftitle}  & ralph waldo emerson dismissed this poet as the jingle man and james russell lowell called him three-fifths genius and two-fifths sheer fudge . & \textbf{Sel.} & \textbf{Ord.} \\
\textbf{\oracletitle}  & sheer this as james two-fifths emerson fudge lowell poet genius waldo called russell the and ralph and him . dismissed jingle three-fifths man 